In [1]:
import auth_context
import importlib

import json
import logging
import os
import sys
import time
from typing import Dict

# import cimgraph.data_profile.rc4_2021 as cim
from cimgraph.data_profile import CIM_PROFILE

from pathlib import Path

import gridappsd.field_interface.agents.agents as agents_mod
from gridappsd.field_interface.agents import CoordinatingAgent, FeederAgent, SwitchAreaAgent, SecondaryAreaAgent
from gridappsd.field_interface.context import ContextManager
from gridappsd.field_interface.interfaces import MessageBusDefinition

import sample_queries as example

cim_profile = CIM_PROFILE.RC4_2021.value

agents_mod.set_cim_profile(cim_profile)

cim = agents_mod.cim

logging.basicConfig(level=logging.DEBUG)
logging.getLogger('goss').setLevel(logging.ERROR)
logging.getLogger('stomp.py').setLevel(logging.ERROR)

_log = logging.getLogger(__name__)

os.environ['GRIDAPPSD_APPLICATION_ID'] = 'dist-sample-app'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'
os.environ['GRIDAPPSD_ADDRESS'] = 'localhost'
os.environ['GRIDAPPSD_PORT'] = '61613'

['/home/d3m614/gridappsd-sample-distributed-app', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/d3m614/.local/lib/python3.8/site-packages', '/home/d3m614/gridappsd-python', '/home/d3m614/CIMHub/src_python', '/home/d3m614/gridappsd-cim-profile', '/home/d3m614/topology-processor', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']


In [2]:
class SampleCoordinatingAgent(CoordinatingAgent):

    def __init__(self, feeder_id, system_message_bus_def, simulation_id=None):
        super().__init__(feeder_id, system_message_bus_def, simulation_id)


class SampleFeederAgent(FeederAgent):

    def __init__(self, upstream_message_bus_def: MessageBusDefinition, downstream_message_bus_def: MessageBusDefinition,
                 feeder_dict: Dict = None, simulation_id: str = None):
        super().__init__(upstream_message_bus_def, downstream_message_bus_def,
                                                feeder_dict, simulation_id)

        
    #TODO remove first four
    def on_measurement(self, headers: Dict, message) -> None:
        _log.debug(f"measurement: {self.__class__.__name__}.{headers.get('destination')}")
        with open("feeder.txt", "a") as fp:
            fp.write(json.dumps(message))
        #print(message)


class SampleSwitchAreaAgent(SwitchAreaAgent):

    def __init__(self, upstream_message_bus_def: MessageBusDefinition, downstream_message_bus_def: MessageBusDefinition,
                 switch_area_dict: Dict = None, simulation_id: str = None):
        super().__init__(upstream_message_bus_def, downstream_message_bus_def,
                                                    switch_area_dict, simulation_id)

    def on_measurement(self, peer, sender, bus, topic, headers, message):
        _log.debug(f"measurement: {self.__class__.__name__}.{headers.get('destination')}")
        with open("switch_area.txt", "a") as fp:
            fp.write(json.dumps(message))
        #print(message)


class SampleSecondaryAreaAgent(SecondaryAreaAgent):

    def __init__(self, upstream_message_bus_def: MessageBusDefinition, downstream_message_bus_def: MessageBusDefinition,
                 secondary_area_dict: Dict = None, simulation_id: str = None):
        super().__init__(upstream_message_bus_def, downstream_message_bus_def,
                                                       secondary_area_dict, simulation_id)

        

    def on_measurement(self, peer, sender, bus, topic, headers, message):
        _log.debug(f"measurement: {self.__class__.__name__}.{headers.get('destination')}")
        with open("secondary.txt", "a") as fp:
            fp.write(json.dumps(message))




In [3]:
def overwrite_parameters(yaml_path: str, feeder_id: str) -> MessageBusDefinition:
    bus_def = MessageBusDefinition.load(yaml_path)
    id_split = bus_def.id.split('.')
    if len(id_split) > 1:
        bus_def.id = feeder_id + '.' +'.'.join(id_split[1:])
    else:
        bus_def.id = feeder_id
    address = os.environ.get('GRIDAPPSD_ADDRESS')
    port = os.environ.get('GRIDAPPSD_PORT')
    if not address or not port:
        raise ValueError("import auth_context or set environment up before this statement.")

    bus_def.conneciton_args['GRIDAPPSD_ADDRESS'] = f"tcp://{address}:{port}"
    return bus_def

In [8]:
feeder_path = Path("simulation.feeder.txt")
if not feeder_path.exists():
    print("Simulation feeder not written, please execute `python run_simulation.py` before this script.")
    sys.exit(0)
simulation_id_path = Path("simulation.id.txt")
if not simulation_id_path.exists():
    print("Simulation id not written, please execute `python run_simulation.py` before executing this script.")
    sys.exit(0)

feeder_id = feeder_path.read_text().strip()
simulation_id = simulation_id_path.read_text().strip()

system_message_bus_def = overwrite_parameters("config_files_simulated/system-message-bus.yml", feeder_id)

#TODO: add dictionary of other message bus definitions or have a functions call
coordinating_agent = SampleCoordinatingAgent(feeder_id, system_message_bus_def)

context = ContextManager.get_context_by_feeder(feeder_id)

# Create feeder level distributed agent
feeder_message_bus_def = overwrite_parameters("config_files_simulated/feeder-message-bus.yml", feeder_id)
feeder = context['data']

#TODO: create access control for agents for different layers
feeder_agent = SampleFeederAgent(system_message_bus_def, feeder_message_bus_def, feeder, simulation_id)
coordinating_agent.spawn_distributed_agent(feeder_agent)


# Get all the attributes of the equipments in the feder area from the model 
#TODO: Uncomment when feeder attributed query working in gridappsd
#print(feeder_agent.feeder_area.get_all_attributes(cim.PowerTransformer))

# create switch area distributed agents
switch_areas = context['data']['switch_areas']
for sw_index, switch_area in enumerate(switch_areas):
    switch_area_message_bus_def = overwrite_parameters(f"config_files_simulated/switch_area_message_bus_{sw_index}.yml", feeder_id)
    print("Creating switch area agent " + str(switch_area['message_bus_id']))
    switch_area_agent = SampleSwitchAreaAgent(feeder_message_bus_def,
                                              switch_area_message_bus_def,
                                              switch_area,
                                              simulation_id)
    coordinating_agent.spawn_distributed_agent(switch_area_agent)

    # Get all the attributes of the equipments in the switch area from the model 
    print(switch_area_agent.switch_area.typed_catalog[cim.ACLineSegment].keys())

    # EXAMPLE 1 - Get phase, bus info about ACLineSegments
    example.get_lines_buses(switch_area_agent.switch_area)

    # EXAMPLE 2 - Get all line impedance data
    example.get_line_impedances(switch_area_agent.switch_area)

    # EXAMPLE 3 - Sort all line impedance by line phase:
    example.sort_impedance_by_line(switch_area_agent.switch_area)

    # Example 4 - Sort all lines by impedance
    example.sort_line_by_impedance(switch_area_agent.switch_area)

    # create secondary area distributed agents
    for sec_index, secondary_area in enumerate(switch_area['secondary_areas']):
        secondary_area_message_bus_def = overwrite_parameters(f"config_files_simulated/secondary_area_message_bus_{sw_index}_{sec_index}.yml", feeder_id)
        print("Creating secondary area agent " + str(switch_area['message_bus_id']))
        secondary_area_agent = SampleSecondaryAreaAgent(switch_area_message_bus_def,
                                                        secondary_area_message_bus_def,
                                                        secondary_area,
                                                        simulation_id)
        if len(secondary_area_agent.secondary_area.addressable_equipment) > 1:
            coordinating_agent.spawn_distributed_agent(secondary_area_agent)

            # Get all the attributes of the equipments in the switch area from the model 
            attributes = secondary_area_agent.secondary_area.get_all_attributes(cim.EnergyConsumerPhase)
            if attributes is not None:
                print('Printing properties for secondary area EnergyConsumerPhase')
                print(attributes)

INFO:goss:Starting thread queue
INFO:goss:Starting thread queue
DEBUG:gridappsd.field_interface.agents.agents:Creating DistributedAgent: SampleFeederAgent
INFO:goss:Starting thread queue
INFO:goss:Starting thread queue
DEBUG:gridappsd.field_interface.agents.agents:subscribing to sim_output on topic /topic/goss.gridappsd.field.simulation.output.615585878._49AD8E07-3BF9-A4E2-CB8F-C3722F837B62
DEBUG:gridappsd.field_interface.agents.agents:Creating DistributedAgent: SampleSwitchAreaAgent
INFO:goss:Starting thread queue


Creating switch area agent _49AD8E07-3BF9-A4E2-CB8F-C3722F837B62.0


INFO:goss:Starting thread queue
DEBUG:gridappsd.field_interface.agents.agents:subscribing to sim_output on topic /topic/goss.gridappsd.field.simulation.output.615585878._49AD8E07-3BF9-A4E2-CB8F-C3722F837B62.0


dict_keys(['_0BBD0EA3-F665-465B-86FD-FC8B8466AD53', '_1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0', '_A04CDFB1-E951-4FC4-8882-0323CD70AE3C', '_FBE26B35-13AB-457D-9795-DF58B28E309D'])

 
 EXAMPLE 1: GET ALL LINE PHASES AND BUSES
line mrid:  _0BBD0EA3-F665-465B-86FD-FC8B8466AD53
line name: 645646
bus 1:  646 _8C58660F-C62C-4903-BE72-22F1255B1E62
bus 2:  645 _0124E881-B82D-4206-BBDF-37D585159872
phase  C :  _94FCAFFF-E5F7-4F97-B697-35C46656EDCA
phase  B :  _CE2AFB05-A1B1-4082-B4EB-2AD8A1D3422D
line mrid:  _1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0
line name: 632645
bus 1:  645 _0124E881-B82D-4206-BBDF-37D585159872
bus 2:  632 _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  C :  _19B6044B-6915-4245-9DD1-6AB0444D42A6
phase  B :  _D9C9CEA6-B3F0-4E02-9911-3DFA192B4D01
line mrid:  _A04CDFB1-E951-4FC4-8882-0323CD70AE3C
line name: 650632
bus 1:  rg60 _673E896A-DCBF-4E43-9924-BEB31C5B6005
bus 2:  632 _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  A :  _2A5A1025-AD81-4BFA-A7F0-ED70F7F43DAA
phase  B :  _2D60B241-DE

IndexError: list index out of range

In [ ]:
# switch_area_agent.switch_area.get_all_attributes(cim.ACLineSegment)
switch_area_agent.switch_area.typed_catalog[cim.ACLineSegment]['_45395C84-F20A-4F5A-977F-B80348256421'].Terminals
switch_area_agent.switch_area.typed_catalog[cim.ACLineSegment]['_FBE26B35-13AB-457D-9795-DF58B28E309D'].Terminals


In [7]:
import json
json.loads(switch_area_agent.switch_area.__dumps__(cim.ACLineSegment))

{'_0BBD0EA3-F665-465B-86FD-FC8B8466AD53': {'mRID': '_0BBD0EA3-F665-465B-86FD-FC8B8466AD53',
  'aliasName': '',
  'description': '',
  'name': '645646',
  'Names': [],
  'AssetDatasheet': '',
  'Assets': [],
  'ConfigurationEvent': [],
  'Controls': [],
  'Location': '_89C21411-D107-454A-93B0-0905893DD27D',
  'Measurements': ['_2d9ed148-d492-4e7d-b9af-81a407e74a59',
   '_4e30ae83-418c-4cfb-99eb-5572e00b4ec4',
   '_5f1dfa9f-e754-471d-9c74-c1a9441e95a3',
   '_84f75532-a514-4355-bf0e-2f8d82c736ae'],
  'OperatingShare': [],
  'PSRType': '',
  'ReportingGroup': [],
  'aggregate': '',
  'inService': '',
  'networkAnalysisEnabled': '',
  'normallyInService': '',
  'AdditionalEquipmentContainer': [],
  'EquipmentContainer': '',
  'Faults': [],
  'OperationalLimitSet': [],
  'UsagePoints': '',
  'BaseVoltage': '_2A158E0C-CD01-4A50-AEBA-59D761FCF15D',
  'SvStatus': [],
  'Terminals': ['_D02B9E99-3557-48F5-B475-4871E47D08F6',
   '_F3C0F2D0-4D28-49B5-A920-37409B99229D'],
  'length': '91.44',
  'b0c

In [ ]:
list(switch_area_agent.switch_area.typed_catalog[cim.ACLineSegment].keys())

In [5]:
print(switch_area_agent.switch_area.get_attributes_query(cim.ACLineSegment))


        PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX cim:  <http://iec.ch/TC57/CIM100#>
        SELECT ?mRID ?name ?BaseVoltage ?Location ?length ?bch ?r ?x ?gch ?b0ch ?r0 ?x0 ?g0ch 
        ?PerLengthImpedance ?WireSpacingInfo
        (group_concat(distinct ?Terminal; separator=";") as ?Terminals) 
        (group_concat(distinct ?Measurement; separator=";") as ?Measurements) 
        (group_concat(distinct ?ACLineSegmentPhase; separator=';') as ?ACLineSegmentPhases)  
        WHERE {          
          ?eq r:type cim:ACLineSegment.
          VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}
          VALUES ?mRID { "_0BBD0EA3-F665-465B-86FD-FC8B8466AD53" 
 "_1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0" 
 "_A04CDFB1-E951-4FC4-8882-0323CD70AE3C" 
 "_FBE26B35-13AB-457D-9795-DF58B28E309D" 
 "_45395C84-F20A-4F5A-977F-B80348256421" 
 "_4C04F838-62AA-475E-AEFA-A63B7C889C13" 
 "_ABF53597-A808-422A-B7EE-552F24D83A5F" 
               } 
        ?eq cim:Equipment.Equi